In [74]:
import pandas as pd
import numpy as np
import pytorch_widedeep as wd
from pytorch_widedeep.preprocessing import WidePreprocessor
from pytorch_widedeep.models import Wide, DeepDense, WideDeep
from pytorch_widedeep.metrics import 

In [51]:
train_path = '../data/interim/train_0.75_0.25.csv'
test_path = '../data/interim/test_0.75_0.25.csv'
genre_path = '../data/raw/movies.csv'
tag_path = '../data/raw/tags.csv'

In [52]:
train, test, genre, tags = pd.read_csv(train_path), pd.read_csv(test_path), pd.read_csv(genre_path), pd.read_csv(tag_path)
train = train.merge(genre).merge(tags.drop('timestamp', axis=1))
test = test.merge(genre).merge(tags.drop('timestamp', axis=1))

In [72]:
train.merge(tags)

(4293, 6)

In [62]:
from pytorch_widedeep.preprocessing import WidePreprocessor, TextPreprocessor, DensePreprocessor

In [79]:
wide_cols = ['genres', 'tag']
crossed_cols = [('genres', 'tag')]
embs = [('userId', 16), ('movieId', 16)]
target = train['rating'].values

In [64]:
preprocess_wide = WidePreprocessor(wide_cols=wide_cols, crossed_cols=crossed_cols)
train_wide = preprocess_wide.fit_transform(train)

In [69]:
# WIDE
preprocess_wide = WidePreprocessor(wide_cols=wide_cols, crossed_cols=crossed_cols)
train_wide = preprocess_wide.fit_transform(train)
test_wide = preprocess_wide.transform(train)

# DEEP
preprocess_deep = DensePreprocessor(embed_cols=embs)
train_deep = preprocess_deep.fit_transform(train)
test_deep = preprocess_deep.transform(test)

In [76]:
wide = Wide(wide_dim=np.unique(train_wide).shape[0], pred_dim=1)

deepdense = DeepDense(hidden_layers=[64,32], 
                      deep_column_idx=preprocess_deep.deep_column_idx,
                      embed_input=preprocess_deep.embeddings_input)

model = WideDeep(wide=wide, deepdense=deepdense)

In [78]:
model.compile(method='regression')

In [ ]:
model.fit(X_wide=train_wide, X_deep=train_deep, target=target, n_epochs=5, batch_size=256, val_split=0.2, 
          warm_up=True, warm_epochs=5, warm_max_lr=0.01)

  0%|          | 0/28 [00:00<?, ?it/s]

Warming up wide for 5 epochs


  0%|          | 0/28 [00:00<?, ?it/s]

Warming up deepdense for 5 epochs


epoch 4: 100%|██████████| 28/28 [00:18<00:00,  1.54it/s, loss=0.513]


In [30]:
text_preprocessor = TextPreprocessor(text_col='genres',min_freq=1, maxlen=10)
train_genre = text_preprocessor.fit_transform(train)
test_genre = text_preprocessor.transform(test)

The vocabulary contains 34 tokens


In [48]:
(train.tag.str.split('|').apply(lambda x: len(x)) == 0).sum()

0

In [42]:
train[train.tag.str.len() == 1]

,userId,movieId,rating,title,genres,tag
1015,211307,733,0.5,"Rock, The (1996)",Action|Adventure|Thriller,R


In [49]:
text_preprocessor = TextPreprocessor(text_col='tag', max_vocab=25, min_freq=0, maxlen=10)
train_tag = text_preprocessor.fit_transform(train)
test_tag = text_preprocessor.transform(test)

The vocabulary contains 25 tokens


ValueError: cannot copy sequence with size 0 to array axis with dimension 10